In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [ ]:
# Initialise Camera
cap = cv2.VideoCapture(0)                 # 0 is the id of the webcam
face_cascade = cv2.CascadeClassifier('Documents/Coding/ML Datasets/haarcascade_frontalface_alt.xml')
skip = 0
face_data = []             #Empty array to store data of every 10th face
dataset_path = 'Documents/Coding/ML Datasets/Facial Data/'    #Path where we will store this.

file_name = input("Enter the name of the person: ")

while True:
    ret,frame = cap.read()                #We read what infomation we are getting from the webcam
    
    if ret == False:
        continue                          #If frame is not captured, try it again
    
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if len(faces)==0:
        continue
        
    faces = sorted(faces,key=lambda f:f[2]*f[3])
    # This will sort the faces using lambda function where key is f[2]*f[3] in a reversed order.
    
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
        offset = 10      
        face_section = frame[y-offset:y+h+offset , x-offset:x+w+offset]    #Slicing
        face_section = cv2.resize(face_section,(100,100))                   #Resizing it into 100 x 100
        # Store every 10th face.    
        skip+=1
        if skip%10==0:
            face_data.append(face_section)
            print(len(face_data))
        
    
    cv2.imshow("Frame",frame)
    cv2.imshow("Face Section",face_section)        
    #Now we'll see 2 frames. 1 with full image and other with only face.
    
    key_pressed = cv2.waitKey(1) & 0xFF            #This will give a 8-bit integer
    if key_pressed == ord('q'):                    #Program will terminate on pressing 'q'.
        break
    #Otherwise we will get the return value.    
#The counter in the output indicates every 10th frame captured. It is incremented after every 10th frame.
cap.release()
cv2.destroyAllWindows()

# Convert our face list array into a numpy array.
face_data = np.asarray(face_data)                  #Converting face_data into a numpy array.
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

#Save this data into file system
np.save(dataset_path+file_name+'.npy',face_data)
print("Data successfully saved at "+dataset_path+file_name+'.npy')

In [ ]:
############ KNN CODE #################
def distance(v1,v2):
    # Euclidean
    return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
    dist=[]
    
    for i in range(train.shape[0]):
        #Get the vector and label
        ix = train[i,:-1]
        iy = train[i, -1]
        #Compute distance from test point
        d = distance(test,ix)
        dist.append([d,iy])
    
    #Sort based on distance and get top k
    dk = sorted(dist, key=lambda x:x[0])[:k]
    #Retrieve only the labels 
    labels = np.array(dk)[:,-1]
    
    #Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    #Find maximum frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]

# Initialise Camera
cap = cv2.VideoCapture(0)                  

# Face Detection
face_cascade = cv2.CascadeClassifier('Documents/Coding/ML Datasets/haarcascade_frontalface_alt.xml')
skip = 0
dataset_path = 'Documents/Coding/ML Datasets/Facial Data/'    #Path where we will store dataset.
face_data = []             # Training data (x for the data)
labels = []                # y for the data

class_id = 0               # Labels for given file. First will have id=0, next as 1 and so on.
names = {}                 # Dictionary for mapping between id and name.

# Data Preparation
for fx in os.listdir(dataset_path):        # Iterating over the directory
#os.listdir() will get all the files which are present in the given folder in the given path (data folder).
    if fx.endswith('.npy'):                #Checking if a file is numpy file which we want.
        # Create a mapping between class_id and name
        names[class_id] = fx[:-4]          #Mapping class_id with name 
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx) #Loading the file in variable data_item
        
        face_data.append(data_item)        
        
        target = class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)
        
face_dataset = np.concatenate(face_data,axis=0)            #x-train
face_labels = np.concatenate(labels,axis=0)                #y-train

print(face_dataset.shape)
print(face_labels.shape)

face_labels = face_labels.reshape((-1,1))                  #For concatenating
trainset = np.concatenate((face_dataset,face_labels),axis=1) 
print(trainset.shape)   #Now we have 30001 features, where first 30000 are features and last one is label. 

# Testing
while True:
    ret,frame = cap.read()
    if False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    if len(faces)==0:
        continue
        
    for face in faces:
        x,y,w,h = face
        
        # Get the face region of interest
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))   #Test frame extracted and reshaped to (100,100)
        
        #Predicted label (output)
        out = knn(trainset,face_section.flatten())      #Linear part of test section required.
        
        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)      #Color of boundary box given in end 
        
    cv2.imshow("Faces",frame)
    key = cv2.waitKey(1) & 0xFF
    
    
    if key == ord('q'):
        break
                    
cap.release()
cv2.destroyAllWindows()                   